
# 🧠 Object Detection using Transfer Learning of CNN Architectures

This practical demonstrates **Transfer Learning** using a pre-trained CNN model.
We will:
a. Load a pre-trained CNN (MobileNetV2)  
b. Freeze lower layers  
c. Add a custom classifier  
d. Train it on a small dataset  
e. Fine-tune and evaluate  


In [ ]:

# Step a: Import Libraries and Load Dataset
import tensorflow as tf
from tensorflow.keras import layers, models

# Load a small dataset (e.g., flower dataset)
dataset_url = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)

# Create training and validation sets
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(160, 160),
    batch_size=32)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(160, 160),
    batch_size=32)


In [ ]:

# Step b: Load Pre-trained CNN (MobileNetV2) and Freeze Lower Layers
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(160, 160, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # freeze convolutional base


In [ ]:

# Step c: Add Custom Classifier
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(5, activation='softmax')  # 5 flower classes
])


In [ ]:

# Step d: Compile and Train the Model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_ds, validation_data=val_ds, epochs=3)


In [ ]:

# Step e: Fine-tune (optional) - unfreeze some layers and re-train with smaller LR
base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history_fine = model.fit(train_ds, validation_data=val_ds, epochs=2)


In [ ]:

# Evaluate final model
loss, acc = model.evaluate(val_ds)
print(f"✅ Final validation accuracy: {acc*100:.2f}%")
